# Reccomendation systems and matrix factorization: simple problem

In [25]:
import numpy as np
from numba import jit, cuda

Goal: predict rating matrix R

In [26]:
@jit(target_backend='cuda')
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [27]:
R = [

     [5,3,0,1],

     [4,0,0,1],

     [1,1,0,5],

     [1,0,0,4],

     [0,1,5,4],
    
     [2,1,3,0],

    ]

R = np.array(R)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])
# Num of Features
K = 20

print(N, ' users, ', M, ' movies, ', K ,' latent features. \n')

 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

print(nP,'\n',nQ)

nR = np.dot(nP, nQ.T)

6  users,  4  movies,  20  latent features. 

[[ 0.40528311  0.5787458   0.88135912  0.44871834  0.48517407  0.09551109
   1.12962691  0.35125426  0.89304336  0.07837037  0.13791497 -0.09055033
   0.7668831  -0.0070091  -0.13562171  0.38553847  0.56794222  0.50413106
   0.57138203  0.85922646]
 [ 0.70836726  0.15776661  0.71533595  0.93595773  0.28124413 -0.19486992
   0.48355318  0.11837877  0.89044169  0.55292566  0.1576084   0.43042768
   0.41643803  0.40693315  0.49891269 -0.06014189  0.87420972  0.09643842
   0.23316486  0.7738602 ]
 [ 0.76598309  0.43867291  0.51845901 -0.12621767 -0.21098551  0.90312489
   0.21990234  0.70151646  0.00638145  0.5642065   0.55903889  0.65830954
   0.24104663  0.86574908  0.96867125  0.64149836 -0.01621438  0.58107388
  -0.10633154 -0.00818067]
 [ 0.18979719  0.25544514  0.58274488 -0.34217576  0.32063832  0.25551403
   0.05308052  0.25112031  0.16328616  0.21406229  0.87821091  0.62812379
   0.31885119  0.84220077  1.007286    0.76970214 -0.095386

In [28]:
print(R)
print(nR)

[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]
 [2 1 3 0]]
[[4.97793385 2.97990968 4.1389725  1.00466031]
 [3.98319016 2.64952107 4.10176002 1.00434345]
 [1.01043085 0.98682729 4.7361644  4.97488419]
 [1.00293497 1.07574321 4.03655865 3.9857411 ]
 [2.49278608 1.00730461 4.98344611 3.98925783]
 [1.98412901 1.01321895 2.99296963 2.49952673]]
